In [2]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import numpy as np

In [ ]:
data = []

# Selenium driver'ını kur
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # İsterseniz arka planda çalıştırabilirsiniz
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Ana sayfayı yükle
url = "https://www.turkiye.gov.tr/hizmetler"
driver.get(url)
time.sleep(3)
soup = BeautifulSoup(driver.page_source, "html.parser")

# Kategorileri bul ve döngüye başla
categories = soup.find_all('ul', class_='serviceCategoriesList')
for category in categories:
    items = category.find_all('li')
    for item in items:
        title_tag = item.find('a')
        if not title_tag: continue  # `a` etiketi yoksa bu adımı atla

        link = title_tag.get('href', '')
        full_link = f"https://www.turkiye.gov.tr{link}" if link.startswith('/') else link
        
        # Kategori ve açıklamasını daha güvenli bir şekilde al
        try:
            category_name, description = item.text.strip().split('\n', 1)
        except ValueError:
            category_name = item.text.strip()
            description = ''

        # Kategori detay sayfasına git
        driver.get(full_link)
        time.sleep(2)
        detail_soup = BeautifulSoup(driver.page_source, "html.parser")
        
        detail = detail_soup.find('ul', class_='agencyServices')
        if not detail: continue

        for service_app in detail.find_all('li', recursive=False):
            if not service_app.find('div', class_='serviceList'):
                continue

            # Kurum bilgilerini al
            service_app_temp = service_app.find(class_='head')
            service_app_title_tag = service_app_temp.find('a') if service_app_temp else None
            service_app_link = service_app_title_tag.get('href', '') if service_app_title_tag else ''
            service_app_link = f"https://www.turkiye.gov.tr{service_app_link}" if service_app_link.startswith('/') else service_app_link
            
            service_app_common_name_tag = service_app_temp.find("h3") if service_app_temp else None
            service_app_common_name = service_app_common_name_tag.text.strip() if service_app_common_name_tag else ''

            services = service_app.find('div', class_='serviceList')
            if not services: continue

            service_list_container = services.find('ul')
            if not service_list_container: continue
            
            # Anahtar mantık burada başlıyor
            for service_group_li in service_list_container.find_all('li', recursive=False):
                group_name_tag = service_group_li.find("h4")
                service_common_name = group_name_tag.text.strip() if group_name_tag else service_app_common_name

                # YAPIYI KONTROL ET: Bu li'nin içinde başka li'ler var mı?
                nested_services = service_group_li.find_all('li')

                # Bu fonksiyon veriyi işlemeyi kolaylaştırır ve kod tekrarını önler
                def process_service_tag(service_tag):
                    if not service_tag:
                        return None

                    elements = {}
                    service_link = service_tag.get('href', '')
                    
                    elements['category'] = category_name.strip()
                    elements['description'] = description.strip()
                    elements["category_full_link"] = full_link
                    elements["service_app_common_name"] = service_app_common_name
                    elements["service_app_link"] = service_app_link
                    elements["service_common_name"] = service_common_name
                    elements["service_name"] = service_tag.text.strip()
                    elements["service_link"] = f"https://www.turkiye.gov.tr{service_link}" if service_link.startswith('/') else service_link
                    elements["service_description"] = service_tag.get('data-description', '').strip()
                    return elements

                if nested_services:
                    # DURUM 1: EVET, İÇİNDE LİSTE VAR (Bu bir grup başlığı)
                    for service_item_li in nested_services:
                        service_tag = service_item_li.find('a')
                        processed_data = process_service_tag(service_tag)
                        if processed_data:
                            data.append(processed_data)
                else:
                    # DURUM 2: HAYIR, İÇİNDE LİSTE YOK (Bu doğrudan bir hizmet linki)
                    service_tag = service_group_li.find('a')
                    processed_data = process_service_tag(service_tag)
                    if processed_data:
                        data.append(processed_data)

# İşlem bittiğinde driver'ı kapat ve veriyi kaydet
driver.quit()
df = pd.DataFrame(data)
df.to_csv('scraped_items_v2.csv', index=False, encoding='utf-8-sig') # Türkçe karakter sorunu için encoding
print("Veri çekme işlemi tamamlandı.")
print(df)